In [184]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,udf,lit,current_timestamp, explode, substring
from pyspark.sql.types import ArrayType,StructType,StructField,StringType,IntegerType,LongType,DoubleType,TimestampType,DecimalType,FloatType
import requests, json
import datetime as datetime
from pyspark.sql import Row
import pytz as pytz

In [185]:
#sc = spark.sparkContext

spark = SparkSession \
  .builder \
  .appName("Movie REST test") \
  .master("local[4]") \
  .config("spark.driver.memory","4G") \
  .getOrCreate()

sc=spark.sparkContext
sc.setLogLevel('INFO')
spark.conf.set("spark.sql.debug.maxToStringFields", 100000)
spark.conf.set("spark.sql.session.timeZone", "UTC+8")

In [186]:
config_schema = StructType([  
  StructField("verb", StringType(), True),
  StructField("url", StringType(), True),
  StructField("start_date", StringType(), True),
  StructField("end_date", StringType(), True),
  StructField("headers", StringType(), True),
  StructField("body", StringType(), True)
])

api_df = spark.read.schema(config_schema).json("apiconfig.json")

api_df.take(1)

[Row(verb=None, url=None, start_date=None, end_date=None, headers=None, body=None)]

In [187]:
verb = "get"

url = "https://boxoffice.tfi.org.tw/api/export"

start_date = "2022/10/02"

end_date = "2022/10/09"

headers = {
    'content-type': "application/json"
}

body = json.dumps({
})

RestApiRequest = Row("verb", "url", "start_date", "end_date", "headers", "body")

tw = pytz.timezone('Asia/Taipei')

In [188]:
def executeRestApi(verb, url, start_date, end_date, headers, body):
    
  res = None

  url = url + "?start={}".format(start_date) + "&end={}".format(end_date)
    
  # Make API request, get response object back, create dataframe from above schema.
  try:
    if verb == "get":
      res = requests.get(url, data=body, headers=headers, timeout=5)
    else:
      res = requests.post(url, data=body, headers=headers)
#  except Exception as e:
#    return e
  except requests.exceptions.RequestException as e:  # This is the correct syntax
      raise SystemExit(e)
  if res != None and res.status_code == 200:
    return json.loads(res.text)
                      
  return None

In [189]:
print("1st stime: ", datetime.datetime.now(tw))

request_df = spark.createDataFrame([
            RestApiRequest(verb, url, start_date, end_date , headers, body)
          ])

# request_df.printSchema()

request_df.take(1)

print("1st etime: ", datetime.datetime.now(tw))

1st stime:  2022-10-11 23:49:36.535533+08:00
1st etime:  2022-10-11 23:49:38.031841+08:00


In [190]:
schema = StructType([  
  StructField("start", StringType(), True),
  StructField("end", StringType(), True),
  StructField("list", ArrayType(
    StructType([
      StructField("country", StringType()),
      StructField("name", StringType()),
      StructField("releaseDate", StringType()),
      StructField("issue", StringType()),
      StructField("produce", StringType()),
      StructField("theaterCount", IntegerType()),
      StructField("tickets", LongType()),
      StructField("ticketChangeRate", FloatType()),
      StructField("amounts", LongType()),
      StructField("totalTickets", LongType()),
      StructField("totalAmounts", LongType())
    ])
  ))
])

udf_executeRestApi = udf(executeRestApi, schema)

In [191]:
print("2nd stime: ", datetime.datetime.now(tw))

request_df = request_df.withColumn("result", \
                udf_executeRestApi(col("verb"), col("url"), col("start_date"), col("end_date"), col("headers"), col("body")))

request_df.show()

print("2nd etime: ", datetime.datetime.now(tw))

2nd stime:  2022-10-11 23:49:38.687867+08:00
+----+--------------------+----------+----------+--------------------+----+--------------------+
|verb|                 url|start_date|  end_date|             headers|body|              result|
+----+--------------------+----------+----------+--------------------+----+--------------------+
| get|https://boxoffice...|2022/10/02|2022/10/09|{content-type -> ...|  {}|{2022-10-02T00:00...|
+----+--------------------+----------+----------+--------------------+----+--------------------+

2nd etime:  2022-10-11 23:49:42.482952+08:00


In [192]:
request_df.printSchema()

root
 |-- verb: string (nullable = true)
 |-- url: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- headers: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- body: string (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- start: string (nullable = true)
 |    |-- end: string (nullable = true)
 |    |-- list: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- releaseDate: string (nullable = true)
 |    |    |    |-- issue: string (nullable = true)
 |    |    |    |-- produce: string (nullable = true)
 |    |    |    |-- theaterCount: integer (nullable = true)
 |    |    |    |-- tickets: long (nullable = true)
 |    |    |    |-- ticketChangeRate: float (nullable = true)
 |    |    |    |-- amounts: long (nullab

In [193]:
print("3rd stime: ", datetime.datetime.now(tw))

request_df.select(explode(col("result.list")).alias("list")) \
    .select(col("list.country"), col("list.name"), \
            substring(col("list.issue"),1,10).alias("issue"), \
            substring(col("list.releaseDate"),1,10).alias("releaseDate"), col("list.tickets")) \
    .sort(col("list.tickets").desc()).show(50, truncate=False)

print("3rd etime: ", datetime.datetime.now(tw))

3rd stime:  2022-10-11 23:49:42.635233+08:00
+--------+------------------------------------------------------------------+--------------------+-----------+-------+
|country |name                                                              |issue               |releaseDate|tickets|
+--------+------------------------------------------------------------------+--------------------+-----------+-------+
|日本    |航海王劇場版 : 紅髮歌姬                                           |薩摩亞商雄讚股份有限|2022-08-13 |11195  |
|美國    |阿凡達 (A版)                                                      |博偉電影股份有限公司|2022-09-23 |10580  |
|日本    |破案天才伽利略：沉默的遊行                                        |威視股份有限公司    |2022-09-23 |6237   |
|韓國    |機密同盟2                                                         |車庫娛樂股份有限公司|2022-09-08 |5675   |
|美國    |捍衛戰士: 獨行俠                                                  |美商美國派拉蒙影片股|2022-05-23 |4762   |
|日本    |新‧超人力霸王                                                     |野獸國股份有限公司  |2022-09-13 |4506  

In [194]:
print("4th stime: ", datetime.datetime.now(tw))

movielist_df = request_df.select(explode(col("result.list")).alias("list"))

# movielist_df.printSchema()

movielist_df.createOrReplaceTempView("list")

spark.sql("""select list.country, 
                    trim(list.name) as movie_name, 
                    trim(substring(list.issue,1,10)) as issue, 
                    substring(list.releaseDate,1,10) as releaseDate,
                    cast(list.tickets as INT) as tickets
             from list ORDER BY tickets desc""") \
         .show(50,truncate=False)

print("4th etime: ", datetime.datetime.now(tw))

4th stime:  2022-10-11 23:49:59.478361+08:00
+--------+------------------------------------------------------------------+--------------------+-----------+-------+
|country |movie_name                                                        |issue               |releaseDate|tickets|
+--------+------------------------------------------------------------------+--------------------+-----------+-------+
|日本    |航海王劇場版 : 紅髮歌姬                                           |薩摩亞商雄讚股份有限|2022-08-13 |11195  |
|美國    |阿凡達 (A版)                                                      |博偉電影股份有限公司|2022-09-23 |10580  |
|日本    |破案天才伽利略：沉默的遊行                                        |威視股份有限公司    |2022-09-23 |6237   |
|韓國    |機密同盟2                                                         |車庫娛樂股份有限公司|2022-09-08 |5675   |
|美國    |捍衛戰士: 獨行俠                                                  |美商美國派拉蒙影片股|2022-05-23 |4762   |
|日本    |新‧超人力霸王                                                     |野獸國股份有限公司  |2022-09-13 |4506  

In [195]:
spark.stop()

In [ ]:
x = requests.get('https://boxoffice.tfi.org.tw/api/export?start=2022/10/02&end=2022/10/09')
print(json.loads(x.text))